In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import TensorDataset, DataLoader
import copy
import pandas as pd

from sklearn.metrics import classification_report

from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, classification_report
)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [63]:
class CNN1D(nn.Module):
    def __init__(self, num_classes=12):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(64, 128, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.AdaptiveAvgPool1d(1)
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.fc(self.conv(x))


In [8]:
from sklearn.metrics import f1_score, classification_report

def compute_f1_scores(y_true, y_pred, model_name="Model"):
    f1_macro = f1_score(y_true, y_pred, average="macro")
    f1_weighted = f1_score(y_true, y_pred, average="weighted")
    f1_micro = f1_score(y_true, y_pred, average="micro")

    print(f"\n=== {model_name} F1 Scores ===")
    print(f"Macro F1    : {f1_macro:.4f}")
    print(f"Weighted F1 : {f1_weighted:.4f}")
    print(f"Micro F1    : {f1_micro:.4f}")

    return {
        "model": model_name,
        "f1_macro": f1_macro,
        "f1_weighted": f1_weighted,
        "f1_micro": f1_micro
    }


In [55]:



df_test= pd.read_csv("/kaggle/input/k-fold-dataset/data_final_v/test.csv")
df_train = pd.read_csv("/kaggle/input/k-fold-dataset/data_final_v/train.csv")



In [56]:
X_train = df_train.drop(columns=['label','label_encoded'
])
y_train = df_train['label_encoded']
X_test = df_test.drop(columns=['label','label_encoded'
])
y_test = df_test['label_encoded']

In [57]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("GPUs available:", torch.cuda.device_count())


GPUs available: 2


In [58]:
def Training(
    model,
    X_train, y_train,
    X_val, y_val,
    epochs=100,
    lr=1e-3,
    patience=10,
    batch_size=256
):
    train_losses, val_losses = [], []

    num_classes = 12

   

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
    y_train = torch.tensor(y_train, dtype=torch.long)
    X_val   = torch.tensor(X_val, dtype=torch.float32).unsqueeze(1)
    y_val   = torch.tensor(y_val, dtype=torch.long)

    train_loader = DataLoader(TensorDataset(X_train, y_train),
                              batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(TensorDataset(X_val, y_val),
                              batch_size=batch_size, shuffle=False)

    best_loss = float("inf")
    best_weights = copy.deepcopy(model.state_dict())
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            loss = criterion(model(xb), yb)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                val_loss += criterion(model(xb), yb).item()

        val_loss /= len(val_loader)
        val_losses.append(val_loss)

        print(f"Epoch {epoch+1}: Train={train_loss:.4f}, Val={val_loss:.4f}")

        if val_loss < best_loss:
            best_loss = val_loss
            best_weights = copy.deepcopy(model.state_dict())
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping")
            break

    model.load_state_dict(best_weights)
    return model, train_losses, val_losses


In [64]:
from sklearn.metrics import classification_report, confusion_matrix

def evaluate_f1_per_class(model, X, y):
    model.eval()
    X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(1).to(device)
    y_tensor = torch.tensor(y, dtype=torch.long).to(device)

    with torch.no_grad():
        y_pred = model(X_tensor).argmax(dim=1)

    report = classification_report(
        y_tensor.cpu().numpy(),
        y_pred.cpu().numpy(),
        output_dict=True,
        zero_division=0
    )

    f1_per_class = {f"class_{i}": report.get(str(i), {}).get("f1-score", 0.0)
                    for i in range(12)}
    macro_f1 = report["macro avg"]["f1-score"]

    return macro_f1, f1_per_class


def evaluate_confusion_matrix(model, X, y):
    model.eval()
    X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(1).to(device)

    with torch.no_grad():
        y_pred = model(X_tensor).argmax(dim=1)

    return confusion_matrix(y, y_pred.cpu().numpy())


In [60]:
def kfold_training(
    model_class,
    model_kwargs,
    X, y,
    k=5,
    epochs=100,
    batch_size=256,
    lr=1e-3,
    patience=10
):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

    best_model, best_f1, best_fold = None, -1, -1
    models, histories = [], []

    X_np = X.values if hasattr(X, "values") else X
    y_np = y.values if hasattr(y, "values") else y

    for fold, (train_idx, val_idx) in enumerate(skf.split(X_np, y_np), 1):
        print(f"\n========== Fold {fold}/{k} ==========")

        model = model_class(**model_kwargs).to(device)

        model, train_loss, val_loss = Training(
            model,
            X_np[train_idx], y_np[train_idx],
            X_np[val_idx], y_np[val_idx],
            epochs, lr, patience, batch_size
        )

        macro_f1, per_class_f1 = evaluate_f1_per_class(
            model, X_np[val_idx], y_np[val_idx]
        )

        cm = evaluate_confusion_matrix(
            model, X_np[val_idx], y_np[val_idx]
        )

        print(f"Fold {fold} | Macro F1: {macro_f1:.4f}")

        if macro_f1 > best_f1:
            best_f1, best_model, best_fold = macro_f1, copy.deepcopy(model), fold

        models.append(model)
        histories.append({
            "train_loss": train_loss,
            "val_loss": val_loss,
            "macro_f1": macro_f1,
            "per_class_f1": per_class_f1,
            "confusion_matrix": cm
        })

    print(f" Best Model → Fold {best_fold} | Macro F1 = {best_f1:.4f}")
    return best_model, models, histories


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)  

best_model, cnn_models, cnn_histories = kfold_training(
    model_class=CNN1D,
    model_kwargs={"num_classes": 12}, 
    X=X_scaled,
    y=y_train,
    k=5,
    epochs=100,
    batch_size=256,
    patience=10
)



========== Fold 1/5 ==========
Epoch 1: Train=0.5015, Val=0.2061
Epoch 2: Train=0.2171, Val=0.1737
Epoch 3: Train=0.1883, Val=0.1554
Epoch 4: Train=0.1722, Val=0.1621
Epoch 5: Train=0.1634, Val=0.1496
Epoch 6: Train=0.1574, Val=0.1320
Epoch 7: Train=0.1516, Val=0.1363
Epoch 8: Train=0.1458, Val=0.1272
Epoch 9: Train=0.1423, Val=0.1382
Epoch 10: Train=0.1366, Val=0.1167
Epoch 11: Train=0.1330, Val=0.1189
Epoch 12: Train=0.1278, Val=0.1161
Epoch 13: Train=0.1237, Val=0.1059
Epoch 14: Train=0.1225, Val=0.1100
Epoch 15: Train=0.1206, Val=0.1049
Epoch 16: Train=0.1161, Val=0.1082
Epoch 17: Train=0.1156, Val=0.1051
Epoch 18: Train=0.1138, Val=0.1004
Epoch 19: Train=0.1116, Val=0.0995
Epoch 20: Train=0.1106, Val=0.0978
Epoch 21: Train=0.1089, Val=0.0992
Epoch 22: Train=0.1098, Val=0.0970
Epoch 23: Train=0.1073, Val=0.0952
Epoch 24: Train=0.1067, Val=0.0969
Epoch 25: Train=0.1055, Val=0.0938
Epoch 26: Train=0.1052, Val=0.0962
Epoch 27: Train=0.1054, Val=0.0915
Epoch 28: Train=0.1034, Val=0.09

In [41]:
print(best_model
     
     )
torch.save(
    best_model.state_dict(),
    "/kaggle/working/cnn_1D_final_model_k_fold.pt"
)

CNN1D(
  (conv): Sequential(
    (0): Conv1d(1, 64, kernel_size=(3,), stride=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(64, 128, kernel_size=(3,), stride=(1,))
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): AdaptiveAvgPool1d(output_size=1)
  )
  (fc): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=128, out_features=64, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=64, out_features=13, bias=True)
  )
)


In [51]:
cnn_model = CNN1D(
    num_classes=12
).to(device)
cnn_model.load_state_dict(
    torch.load(
        r"/kaggle/working/cnn_1D_final_model_k_fold.pt",
        map_location=device
    )
)

<All keys matched successfully>

In [55]:

y_pred_test = predict_cnn(cnn_model, X_test)
print(y_pred_test)

[ 9  4  4 ...  9 10  9]


In [56]:
y_pred_test

array([ 9,  4,  4, ...,  9, 10,  9])

In [59]:
print("=== Test Set Evaluation ===")
evaluate_model(cnn_model, X_test, y_test, class_names=[
    "Benign", "DNS", "LDAP", "MSSQL", "NTP", "NetBIOS",
    "Portmap", "SNMP", "Syn", "TFTP", "UDP", "UDPLag", "WebDDoS"
])

=== Test Set Evaluation ===
Accuracy      : 0.9600
Macro F1      : 0.6759
Weighted F1   : 0.9645
Micro F1      : 0.9600

Classification Report:

              precision    recall  f1-score   support

      Benign       1.00      0.96      0.98     19566
         DNS       0.37      0.29      0.33       734
        LDAP       0.47      0.63      0.54       669
       MSSQL       0.84      0.93      0.88      2947
         NTP       1.00      0.99      0.99     24274
     NetBIOS       0.52      0.78      0.62       248
     Portmap       0.12      0.18      0.15       137
        SNMP       0.50      0.54      0.52       543
         Syn       1.00      0.99      0.99      9875
        TFTP       1.00      0.99      1.00     19784
         UDP       0.92      0.96      0.94      5702
      UDPLag       0.92      0.74      0.82      1786
     WebDDoS       0.01      0.90      0.02        10

    accuracy                           0.96     86275
   macro avg       0.67      0.76      0.68

{'accuracy': 0.9600231816864677,
 'f1_macro': 0.6759355111878775,
 'f1_weighted': 0.9645481435278768,
 'f1_micro': 0.9600231816864677}

In [58]:
df = pd.DataFrame({'prediction': y_pred_test})

df.to_csv('predictions.csv', index=False)

print("✅ CSV file created: predictions.csv")


✅ CSV file created: predictions.csv
